## Set the source and target workspace for the items to be copied to. 

In [9]:
#Set variables - use the workspace id's, not the name
source_workspace = ""
target_workspace = ""

#Get a bearer token for an active user
token = "Bearer "

## Get the items in the source workspace

In [ ]:
# Get the items in the current workspace - which is Fabric Managed VNet F-SKU

import requests

# Define the API endpoint
url = f'https://api.fabric.microsoft.com/v1/workspaces/{source_workspace}/items'

# Define the headers for the API request
headers = {
    'Authorization': token,
    'Content-Type': 'application/json'
}

# Define the payload for moving the workspace


# Make the API request
response = requests.get(url, headers=headers)
print(response)

# Check if the request was successful
if response.status_code in [200, 201]:
    # Parse the response as JSON
    data = response.json()
    print(data)
else:
    print(f"Request failed with status code {response.status_code}")

# Loop through the list of objects and get the lakehouses, then create a new lakeshouses from those in the target workspace

In [ ]:
#Loop through the object and get the lakehouses, then create a new one in the new workspace

# Get the number of items in the 'value' list
num_values = len(data['value'])

# Now 'num_values' contains the number of items in the 'value' list
print(num_values)

# Loop through each item in the 'value' list
for item in data['value']:
    # Check if the 'type' field is 'Report'
    name = item['displayName']
    ##if item['type'] == 'Lakehouse' and 'dataverse' in item['displayName']:
    if item['type'] == 'Lakehouse': 
        # Do something with the item
        # Define the API endpoint
        url = f'https://api.fabric.microsoft.com/v1/workspaces/{target_workspace}/lakehouses'
        payload = {
            "displayName": f"{name}",
            "description": "A Lakehouse created  by Rest API"
        }

        print("New Lakehouse being created is: " + name)

        response = requests.post(url, headers=headers, json=payload)
        print(response)
        # Check if the request was successful
        if response.status_code in [200, 201]:
            # Parse the response as JSON
            temp_data = response.json()
            print(temp_data)
        
        if response.status_code == 400:
            # Parse the response as JSON
            temp_data = response.json()
            print(temp_data)
            if temp_data['errorCode'] == "ItemDisplayNameAlreadyInUse":
                print("Lakehouse already exists, no issues")


# Next we get the list of tables in the lakehouse and add the tables with the Dataverse shortcut to the target lakehouse

In [ ]:
# Next we will get the list of tables in each lakehouse and add the tables with the Dataverse shortcut to the target lakehouses


# Loop through each item in the 'value' list
for item in data['value']:

    name = item['displayName']
    print("Name is: " +name)

    #If it is a Dataverse lakehouse get the list of tables
    if item['type'] == 'Lakehouse' and 'dataverse' in item['displayName']:
        lakehouseID = item['id']
        print("Lakehouse ID: " + lakehouseID)
        url_lakehouse_tables = f"https://api.fabric.microsoft.com/v1/workspaces/{source_workspace}/lakehouses/{lakehouseID}/tables"

        headers = {
         'Authorization': token,
            'Content-Type': 'application/json'
        }

        response = requests.get(url_lakehouse_tables, headers=headers)
        print(response)
        if response.status_code in [200, 201]:
            # Parse the response as JSON
            table_data = response.json()
            print(table_data)
        
        #For each table get the dataverse shortcut details
        for item in table_data['data']:
            table_name = item['name']
            print("Now getting Dataverse table details: ")
            
            url_get_dataverse_tables = f"https://api.fabric.microsoft.com/v1/workspaces/{source_workspace}/items/{lakehouseID}/shortcuts/Tables/{table_name}"

            response = requests.get(url_get_dataverse_tables, headers=headers)

            if response.status_code == 200:
                dataverse_table_data = response.json()
                print("Dataverse details are: ")
                print(dataverse_table_data)

            else:
                print(f"Request failed with status code {response.status_code}")

            # Now get the lakehouse ID's for the target workspace so we can populate those lakehouses
            url_get_lakehouse_id = f"https://api.fabric.microsoft.com/v1/workspaces/{target_workspace}/items"

            response = requests.get(url_get_lakehouse_id, headers=headers)

            if response.status_code in [200, 201]:
            # Parse the response as JSON
                targetitems = response.json()
            # Loop through each item in the 'value' list
                for item in targetitems['value']:
                # Check if the 'type' field is 'Lakehouse'
                    name = item['displayName']
                    if item['type'] == 'Lakehouse':
                    ##if it is a lakehouse then get the table information 
                        targetlakehouseID = item['id']
                        url_create_lakehouse_tables = f"https://api.fabric.microsoft.com/v1/workspaces/{target_workspace}/items/{targetlakehouseID}/shortcuts"
                        print("url to create lakehouse tables" + url_create_lakehouse_tables)
                        #Populate the target lakehouse shortcut with the source dataverse shortcut details
                        connectionID = dataverse_table_data['target']['dataverse']['connectionId']
                        deltaLakeFolder = dataverse_table_data['target']['dataverse']['deltaLakeFolder']
                        environmentDomain = dataverse_table_data['target']['dataverse']['environmentDomain']
                        tableName = dataverse_table_data['target']['dataverse']['tableName']
                        name = dataverse_table_data['name']


                        #Now create the payload to create the new tables
                        create_dataverse_tables_payload = {
                            "path": "Tables",
                            "name": f"{name}",
                            "target": {
                                "dataverse": {
                                "connectionId": f"{connectionID}",
                                "deltaLakeFolder": f"{deltaLakeFolder}",
                                    "environmentDomain": f"{environmentDomain}",
                                    "tableName": f"{tableName}"
                                }
                            }
                            }

                        print("============")
                        print(create_dataverse_tables_payload)
                        print("============")

                        response = requests.post(url_create_lakehouse_tables, headers=headers, json=create_dataverse_tables_payload)

                        if response.status_code in [200, 201]:
                            response = response.json()
                            print("Dataverse create tables details are: ")
                            print(response)

                        else:
                            print(f"Request to create the Dataverse shortcuts failed with status code {response.status_code}")




# This section will get the report and report definitions from the source database and then publish them to the target workspace

In [ ]:
#This section will get the report and report definitions from the source database and then publish them to the target workspace.connectionID

#Loop through the object and get the reports, then create a new one in the new workspace




# Loop through each item in the source 'value' list
for item in data['value']:
    # Check if the 'type' field is 'Report'
    if item['type'] == 'Report':

        reportId = item['id']
        name = item['displayName']
        description = item['description']


        #Get the source report definition
        print("Getting the report definition ")
        url = f'https://api.fabric.microsoft.com/v1/workspaces/{source_workspace}/reports/{reportId}/getDefinition'


        response = requests.post(url, headers=headers)
        print(response)
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response as JSON
            print("report definition retrieved successfully")
            print()
            report_data = response.json()
            #print(report_data)
        else:
            # Parse the response as JSON
            temp_data = response.json()
            print(temp_data)

        ## The 2 paths below are both needed to populate the report definition

        path1 = report_data['definition']['parts'][0]['path']
        print("path1 is")
        print(path1)
        payload1 = report_data['definition']['parts'][0]['payload']
        payloadType1 = report_data['definition']['parts'][0]['payloadType']

        path2 = report_data['definition']['parts'][2]['path']
        print("path2 is")
        print(path2)
        payload2 = report_data['definition']['parts'][2]['payload']
        payloadType2 = report_data['definition']['parts'][2]['payloadType']

        #print("Payload 0 is" + payload1)
        #print("Payload 2 is " +payload2)

        #This payload is to update the existing report


        update_payload = {
            "definition": {
                "parts": [
                {
                    "path": f"{path1}",
                    "payload": f"{payload1}",
                    "payloadType": "InlineBase64"
                },
                {
                    "path": f"{path2}",
                    "payload": f"{payload2}",
                    "payloadType": "InlineBase64"
                }
            ]
            }
        }

        #This is to publish a report that doesn't exist in the target workspace
        url_to_publish_report_definition = f'https://api.fabric.microsoft.com/v1/workspaces/{target_workspace}/reports'

        payload = {
            "displayName": f"{name}",
            "description": "this is an updated description",
            "definition": {
                "parts": [
                {
                    "path": f"{path1}",
                    "payload": f"{payload1}",
                    "payloadType": "InlineBase64"
                },
                {
                    "path": f"{path2}",
                    "payload": f"{payload2}",
                    "payloadType": "InlineBase64"
                }
            ]
            }
        }
        
        #This will get the target workspace items so we can see if the report already exists further below
        url_get_target_workspace_items = f"https://api.fabric.microsoft.com/v1/workspaces/{target_workspace}/items"

        target_response = requests.get(url_get_target_workspace_items, headers=headers)

        if target_response.status_code in [200, 201]:
            # Parse the response as JSON
            targetitems = target_response.json()
            print (targetitems)
        else:
            print('The call failed ')
            print(target_response.status_code)


        # Loop through each item in the source 'value' list and the see if the report already exists in the target workspace
        for item in data['value']:
            # Check if the 'type' field is 'Report'
            if item['type'] == 'Report':
                name = item['displayName']
                description = item['description']
                # Check if item is in the list
                if any(obj['displayName'] == name for obj in targetitems['value']):
                    print(f'{name} is in the list')
                    #Loop through the target items and find the exact report that matches the source so we can update the definition
                    for item in targetitems['value']:
                        if item['displayName'] == name:
                            print('target report id is ')
                            print(item['id'])
                            reportID = item['id']
                            print
                            update_url = f'https://api.fabric.microsoft.com/v1/workspaces/{target_workspace}/reports/{reportID}/updateDefinition'
                            report_update_response = requests.post(update_url, headers=headers, json=update_payload)
                            if report_update_response.status_code in [200, 201,202]:
                                # Parse the response as JSON
                                print("Report update status code is: ")
                                print(response.status_code)
                            else:
                                # Parse the response as JSON
                                temp_data = report_creation_response.json()
                                print("Report update not successful")
                                print(temp_data)
                else: 
                    #This is to create a new report because it doesn't exist
                    #print("sending the request to create the new report. Payload looks like: " + payload)
                    report_creation_response = requests.post(url_to_publish_report_definition, headers=headers, json=payload)
                    # Check if the request was successful
                    if response.status_code in [200, 201,202]:
                        # Parse the response as JSON
                        report_data = report_creation_response.json()
                        print("Report data is ")
                        print(report_data)
                        print("Response status code is: ")
                        print(response.status_code)
                    else:
                        # Parse the response as JSON
                        temp_data = report_creation_response.json()
                        print("Report not created")
                        print(temp_data)




## This section will rebind the reports to the new Lakehouse tables in the target workspace - Make sure the default semantic model for the lakehouse in the target workspace has been populated with the new tables or the report won't launch.

In [ ]:
# Get the items in the target workspace - which is Fabric Managed VNet F-SKU


# Define the API endpoint
url = f'https://api.fabric.microsoft.com/v1/workspaces/{target_workspace}/items'

# Define the headers for the API request
headers = {
    'Authorization': token,
    'Content-Type': 'application/json'
}

# Define the payload for moving the workspace


# Make the API request
response = requests.get(url, headers=headers)
print(response)

# Check if the request was successful
if response.status_code in [200, 201, 202]:
    # Parse the response as JSON
    target_data = response.json()
    print(target_data)
else:
    print(f"Request failed with status code {response.status_code}")

# Get the number of items in the 'value' list
num_values = len(target_data['value'])

# Now 'num_values' contains the number of items in the 'value' list
print(num_values)

# Loop through each item in the 'value' list
for item in target_data['value']:
    if item['type'] == 'SemanticModel': 
        name = item['displayName']
        semanticModel_id = item['id']
        print('semantic model id is ' + semanticModel_id)

for item in target_data['value']:
    if item['type'] == 'Report': 
        report_name = item['displayName']
        report_id = item['id']
        print('report name is ' + report_name)
        print('report id is ' + report_id)

        url_rebind = f"https://api.powerbi.com/v1.0/myorg/groups/{target_workspace}/reports/{report_id}/Rebind"	
        payload = {
            'datasetId' : f"{semanticModel_id}"
        }	
       
        print('payload is ' + str(payload))
        print('url rebind is ' + url_rebind)
		
        rebind_response = requests.post(url_rebind, headers=headers, json=payload)

        if rebind_response.status_code in [200, 201]:
            print("Successfully rebinded reports")
            print(response)
        else:
            print(rebind_response.status_code)


 

## This section will get the notebooks from the source workspace and their definition, then publish them to the target workspace. 


In [ ]:
#This section will get the report and report definitions from the source database and then publish them to the target workspace.connectionID
import time
#Loop through the object and get the lakehouses, then create a new one in the new workspace

# Get the number of items in the 'value' list
num_values = len(data['value'])

# Now 'num_values' contains the number of items in the 'value' list
print(num_values)

# Loop through each item in the 'value' list
for item in data['value']:
    # Check if the 'type' field is 'Report'
    name = item['displayName']
    description = item['description']
    if item['type'] == 'Notebook':
        # Do something with the item
        # Define the API endpoint
        notebookId = item['id']
        notebookName = item['displayName']
        notebookDescription = item['description']
        print('The notebook name is ' +notebookName)
        print("Getting the notebook definition ")
        url = f'https://api.fabric.microsoft.com/v1/workspaces/{source_workspace}/notebooks/{notebookId}/getDefinition'

        print('The url to get the notebooks is ' +url)
        notebook_response = requests.post(url, headers=headers)
        #print('The notebook response is' + notebook_response)
        
        # Check if the request was successful
        if notebook_response.status_code in [200, 201, 202]:
            # Parse the response as JSON
            print("initial notebook call completed")
            notebook_data1 = notebook_response.json()
            print('Sleeping for 10 seconds so that notebook definition can be generated')
            time.sleep(10)
            location_header = notebook_response.headers.get('Location')
            print('the location header field is ' + location_header)
            print('calling the second url to see if notebook definition is ready ')
            notebook_response2 = requests.get(location_header, headers=headers)
            if notebook_response2.status_code in [200, 201, 202]:
                notebooktemp_data = notebook_response2.json()
                if notebooktemp_data['status'] == 'Succeeded':
                    print('second notebook call successful')
                    location2 = notebook_response2.headers.get('Location')
                    print('now getting the definition with the third  call ')
                    notebook_response3 = requests.get(location2, headers=headers)

                    if notebook_response3.status_code in [200, 201, 202]:
                            notebook_response4 = notebook_response3.json()
                            print("Third call with definition is successful: ")
                            print(notebook_response4)
                            notebook_definition = notebook_response4['definition']
                            #notebook_path = notebook_response4['definition']['parts'][0]['path']
                            #notebook_payload = notebook_response4['definition']['parts'][0]['payload']
                            #notebookpayloadType = notebook_response4['definition']['parts'][0]['payloadType']



                            #Now create the notebooks in the new environment
                            target_workspace_create_notebook_url = f'https://api.fabric.microsoft.com/v1/workspaces/{target_workspace}/notebooks'
                            create_notebook_payload = {
                                    "displayName": f"{notebookName}",
                                    "description": f"{notebookDescription}",
                                    "definition": notebook_definition
                                   }

                            print ('payload is ')
                            print(create_notebook_payload)
                            print("target_workspace_create_notebook_url = " + target_workspace_create_notebook_url)
                            notebook_create = requests.post(target_workspace_create_notebook_url, headers=headers, json=create_notebook_payload)
                             
                            if notebook_create.status_code in [200, 201, 202]:
                                 notebook_create_response_data = notebook_create.json()
                                 print('Notebook creation successful ')
                                 print(notebook_create_response_data)

                            else:
                               # Parse the response as JSON
                                temp_data = notebook_create.json()
                                print(notebook_create.status_code)
                                print(temp_data)
        else:
            # Parse the response as JSON
            temp_data = notebook_response.json()
            print("call failed")
            print(temp_data)


        
